In [ ]:
import matplotlib
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import os
import umap
import datashader as ds
import colorcet as cc
import igraph
import tqdm
from scipy import sparse
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans
from scipy.spatial import Delaunay

from matplotlib.collections import PolyCollection
from matplotlib.colors import ListedColormap

from dredFISH.Analysis import TissueGraph
from dredFISH.Visualization import Viz
from dredFISH.Utils.__init__plots import * 
from dredFISH.Utils import powerplots
from dredFISH.Utils import miscu
from dredFISH.Utils import tmgu

import importlib
importlib.reload(Viz)
importlib.reload(TissueGraph)

#### Load data

In [ ]:
respath = '/bigstore/GeneralStorage/fangming/projects/dredfish/figures/'

In [ ]:
basepth = '/bigstore/GeneralStorage/Data/dredFISH/Dataset1-t5'
!ls -alhtr $basepth
!head $basepth"/TMG.json"

In [ ]:
df = pd.read_csv(
    os.path.join(basepth, "default_analysis.csv"), index_col=0)
df

In [ ]:
TMG = TissueGraph.TissueMultiGraph(basepath=basepth, 
                                   redo=False, # load existing 
                                  )

In [ ]:
# spatial coordinates
layer = TMG.Layers[0]
XY = layer.XY
x, y = XY[:,0], XY[:,1]
###
x, y = y, x # a temporary hack
###

cells = layer.adata.obs.index.values

N = layer.N
# measured basis
ftrs_mat = layer.feature_mat

# umap_mat = umap.UMAP(n_neighbors=30, min_dist=0.1).fit_transform(ftrs_mat)



In [ ]:
# types
types = df['type_r0.1'].values
# local env
# env_mat = get_local_typeabundance(layer.SG, types)

In [ ]:
dd = Delaunay(XY)
EL = np.zeros((dd.simplices.shape[0]*3,2),dtype=np.int64)
for i in range(dd.simplices.shape[0]): 
    EL[i*3,  :] = [dd.simplices[i,0], dd.simplices[i,1]]
    EL[i*3+1,:] = [dd.simplices[i,0], dd.simplices[i,2]]
    EL[i*3+2,:] = [dd.simplices[i,1], dd.simplices[i,2]]
    
tSG = igraph.Graph(n=len(XY), edges=list(zip(EL[:,0], EL[:,1])), directed=False).simplify()

In [ ]:
tSG_adj = tSG.get_adjacency_sparse()
tSG_recon = tmgu.adjacency_to_igraph(tSG_adj, directed=False)

In [ ]:
dgr = layer.SG.indegree() # same as out
dgr_t = tSG.indegree()
dgr_tr = tSG_recon.indegree()

fig, ax = plt.subplots()
sns.histplot(data=dgr, element='step', bins=np.arange(0, 21), ax=ax)
ax.set_xlabel('Number of Delaunay neighbors')
ax.set_ylabel('Number of cells')
plt.show()

fig, ax = plt.subplots()
sns.histplot(data=np.array(dgr_t), element='step', bins=np.arange(0, 21), ax=ax)
ax.set_xlabel('Number of Delaunay neighbors')
ax.set_ylabel('Number of cells')
plt.show()

fig, ax = plt.subplots()
sns.histplot(data=dgr_tr, element='step', bins=np.arange(0, 21), ax=ax)
ax.set_xlabel('Number of Delaunay neighbors')
ax.set_ylabel('Number of cells')
plt.show()

In [ ]:
polys = XY[dd.convex_hull]
polys2 = TMG.Geoms[0]['poly'] #) #) #[0]

xcoords = polys[:,:,0]
xmin, xmax = np.min(xcoords), np.max(xcoords)

ycoords = polys[:,:,1]
ymin, ymax = np.min(ycoords), np.max(ycoords)

fig, ax = plt.subplots(figsize=(20,20))
ax.scatter(XY[::10,0], XY[::10,1], s=0.1, color='k')
p2 = LineCollection(polys2, color='C1', linewidth=0.2)
p = LineCollection(polys)
ax.add_collection(p)
ax.add_collection(p2)
ax.set_aspect('equal') #, 'box')
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
plt.show()

In [ ]:
polys = XY[dd.convex_hull]
polys2 = TMG.Geoms[0]['poly'] #) #) #[0]

xcoords = polys[:,:,0]
xmin, xmax = np.min(xcoords), np.max(xcoords)

ycoords = polys[:,:,1]
ymin, ymax = np.min(ycoords), np.max(ycoords)

fig, ax = plt.subplots(figsize=(15,15))
ax.scatter(XY[::,0], XY[::,1], s=0.2, color='k')
p2 = LineCollection(polys2, color='C1', linewidth=0.2)
p = LineCollection(polys)
ax.add_collection(p)
ax.add_collection(p2)
ax.set_aspect('equal') #, 'box')
ax.set_xlim(-2000,0) #xmin, xmax)
ax.set_ylim(-1000,0) #ymin, ymax)
ax.axis('off')

plt.show()